In [1]:
! pip install kaggle

Upload your Kaggle.json file


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Sentiment dataset

In [ ]:
#!/bin/bash
!kaggle datasets download hbaflast/french-twitter-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/hbaflast/french-twitter-sentiment-analysis
License(s): DbCL-1.0
 68% 33.0M/48.3M [00:00<00:00, 66.0MB/s]
100% 48.3M/48.3M [00:00<00:00, 83.7MB/s]


In [3]:
# extracting the comrpessed dataset (csv inside the zip file)

from zipfile import ZipFile
dataset = '/content/french-twitter-sentiment-analysis.zip' # path of the dataset

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/french-twitter-sentiment-analysis.zip'

Importing the dependencies

In [4]:
import numpy as np
import pandas as pd
import regex as re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# printing the stopwords in french

print(stopwords.words('french'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

Data Processing

In [5]:
# loading the data from the csv file to pandas dataframe
twitter_data = pd.read_csv('/content/french_tweets.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/french_tweets.csv'

In [ ]:
# checking number of rows and columns

twitter_data.shape

(1526724, 2)

In [ ]:
twitter_data.head()

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


In [ ]:
# counting the number of missingvalues
twitter_data.isnull().sum()

,0
label,0
text,0


In [ ]:
# checking the distribution of target column
twitter_data['label'].value_counts()

,count
label,
0,771604
1,755120


Stemming

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

  pattern = re.compile(r'[^\p{L}\s]')
  stemmed_content = pattern.sub(' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('french')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [ ]:
def preprocessing2(content):


  stemmed_content = re.sub(r'[^a-zA-Zà-ÿ\s]', '', content)
  stemmed_content = stemmed_content.lower()


  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(preprocessing2)

In [12]:
twitter_data.head()

,label,text,stemmed_content
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi...",awww bummer devrai avoir david carr troisièm j...
1,0,Est contrarié qu'il ne puisse pas mettre à jou...,contrarié puiss mettr jour facebook télémaigna...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...,plongé plusieur foi ball a réussi économis res...
3,0,Tout mon corps a des démangeaisons et comme si...,tout corp a démangeaison comm si feu
4,0,"Non, il ne se comporte pas du tout. je suis en...",non comport tout colèr pourquoi ici parc peux ...


In [ ]:
# Export whole dataset to the local machine as CSV File
csv_file = 'twitter_stemmed_data.csv'  # Name your file
twitter_data.to_csv(csv_file, index=False)

from google.colab import files

# Trigger a download to your local machine
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['label'].values

Splitting data between training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
# Converting the textual data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

Training the Machine Learning Model

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Accuracy score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print(training_data_accuracy)
print(test_data_accuracy)

0.776067870824699
0.7670995103898869


Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))